## Exemplo1 : Agregação

In [21]:
!pip install pyarrow --quiet

import pandas as pd
import numpy as np

# Técnica 1: Armazenamento Colunar
def armazenamento_colunar(dados):
    df = pd.DataFrame(dados)
    df_colunar = df.to_dict('list')
    return df_colunar

# Técnica 2: Processamento Vetorial
def processamento_vetorial(dados):
    produtos = dados['Produto'].unique()
    resultado = {}
    for produto in produtos:
        quantidades = dados[dados['Produto'] == produto]['Quantidade'].values
        resultado[produto] = np.sum(quantidades)
    return resultado



# Técnica 3: Uso de Disco
def uso_de_disco(dados, arquivo):
    pd.DataFrame(dados).to_csv(arquivo, index=False)
    dados_lidos = pd.read_csv(arquivo)
    return dados_lidos

# Técnica 4: Suporte a Formatos de Arquivo
def suporte_a_formatos(dados, arquivo):
    pd.DataFrame(dados).to_parquet(arquivo, index=False)
    dados_lidos = pd.read_parquet(arquivo)
    return dados_lidos

# Gera o arquivo CSV
dados = [
    {'Produto': 'Arroz', 'Quantidade': 10},
    {'Produto': 'Feijão', 'Quantidade': 20},
    {'Produto': 'Arroz', 'Quantidade': 15},
    {'Produto': 'Feijão', 'Quantidade': 30},
    {'Produto': 'Açúcar', 'Quantidade': 5},
    {'Produto': 'Açúcar', 'Quantidade': 10}
]

arquivo_csv = "produtos.csv"
pd.DataFrame(dados).to_csv(arquivo_csv, index=False)
print('Dataset Original')
print(pd.DataFrame(dados, columns=['Produto','Quantidade']))
arquivo_csv = "produtos.csv"
pd.DataFrame(dados).to_csv(arquivo_csv, index=False)
print('\n')
# Lê o arquivo CSV e agrega os dados por produto
df = pd.read_csv(arquivo_csv)
df_agregado = df.groupby('Produto')['Quantidade'].sum().reset_index()

# Armazenamento Colunar
df_colunar = armazenamento_colunar(df_agregado)
print("Armazenamento Colunar:", df_colunar)
print('\n')
# Processamento Vetorial
resultado = processamento_vetorial(df_agregado)
print("Processamento Vetorial:", resultado)
print('\n')
# Uso de Disco
dados_lidos = uso_de_disco(df_agregado, "agregado.csv")
print("Uso de Disco(CSV):", dados_lidos)
print('\n')
# Suporte a Formatos de Arquivo
print('Dataset Final')
dados_lidos = suporte_a_formatos(df_agregado, "agregado.parquet")
print("Suporte a Formatos(Parquet):", dados_lidos)

Dataset Original
  Produto  Quantidade
0   Arroz          10
1  Feijão          20
2   Arroz          15
3  Feijão          30
4  Açúcar           5
5  Açúcar          10


Armazenamento Colunar: {'Produto': ['Arroz', 'Açúcar', 'Feijão'], 'Quantidade': [25, 15, 50]}


Processamento Vetorial: {'Arroz': 25, 'Açúcar': 15, 'Feijão': 50}


Uso de Disco(CSV):   Produto  Quantidade
0   Arroz          25
1  Açúcar          15
2  Feijão          50


Dataset Final
Suporte a Formatos(Parquet):   Produto  Quantidade
0   Arroz          25
1  Açúcar          15
2  Feijão          50


## Exemplo2 : Join mostrando o valor ganho com cada produto

In [20]:
!pip install pyarrow --quiet
import pandas as pd
import numpy as np

# Técnica 1: Armazenamento Colunar
def armazenamento_colunar(dados):
    df = pd.DataFrame(dados)
    df_colunar = df.to_dict('list')
    return df_colunar

# Técnica 2: Processamento Vetorial
def processamento_vetorial(dados):
    produtos = dados['Produto'].unique()
    resultado = {}
    for produto in produtos:
        
        quantidades = dados[dados['Produto'] == produto]['Quantidade'].values
        preco = dados[dados['Produto'] == produto]['Preco'].values[0]
        resultado[produto] = np.sum(quantidades * preco)
    return resultado

# Técnica 3: Uso de Disco
def uso_de_disco(dados, arquivo):
    pd.DataFrame(dados).to_csv(arquivo, index=False)
    dados_lidos = pd.read_csv(arquivo)
    return dados_lidos

# Técnica 4: Suporte a Formatos de Arquivo
def suporte_a_formatos(dados, arquivo):
    
    pd.DataFrame(dados).to_parquet(arquivo, index=False)
    dados_lidos = pd.read_parquet(arquivo)
    return dados_lidos

# --- Execução Principal ---

# Lê as tabelas conforme os nomes dos arquivos enviados
# Fonte 2 contém os preços 
produtos = pd.read_csv('produto.csv') 
# Fonte 1 contém as quantidades vendidas 
vendas = pd.read_csv('vendas.csv')  
print('Tabela Produtos')
print(produtos)
print('\n')
print('Tabela Vendas')
print(vendas)
print('\n')
# Realiza o Join
# O sufixo não será aplicado em 'Preco' ou 'Quantidade' pois eles são únicos em suas tabelas
resultado = pd.merge(vendas, produtos, on='Produto')

# 1. Armazenamento Colunar
df_colunar = armazenamento_colunar(resultado)
print("Armazenamento Colunar:", df_colunar)

# 2. Processamento Vetorial
resultado_pv = processamento_vetorial(resultado)
print("\nProcessamento Vetorial (Total por Produto):", resultado_pv)

# 3. Uso de Disco
dados_lidos_csv = uso_de_disco(resultado, "resultado.csv")
print("\nUso de Disco (CSV) - Primeiras linhas:\n", dados_lidos_csv.head(7))

# 4. Suporte a Formatos de Arquivo
try:
    dados_lidos_pq = suporte_a_formatos(resultado, "resultado.parquet")
    print("\nSuporte a Formatos (Parquet) - Primeiras linhas:\n", dados_lidos_pq.head(7))
except ImportError:
    print("\nSuporte a Formatos: Instale 'pyarrow' para testar Parquet.")

Tabela Produtos
  Produto  Preco
0   Arroz    2.5
1  Feijão    3.0
2  Açúcar    1.5


Tabela Vendas
  Produto  Quantidade
0   Arroz          10
1  Feijão          20
2   Arroz          15
3  Açúcar           5
4  Feijão          30
5  Açúcar          10


Armazenamento Colunar: {'Produto': ['Arroz', 'Arroz', 'Feijão', 'Feijão', 'Açúcar', 'Açúcar'], 'Quantidade': [10, 15, 20, 30, 5, 10], 'Preco': [2.5, 2.5, 3.0, 3.0, 1.5, 1.5]}

Processamento Vetorial (Total por Produto): {'Arroz': 62.5, 'Feijão': 150.0, 'Açúcar': 22.5}

Uso de Disco (CSV) - Primeiras linhas:
   Produto  Quantidade  Preco
0   Arroz          10    2.5
1   Arroz          15    2.5
2  Feijão          20    3.0
3  Feijão          30    3.0
4  Açúcar           5    1.5
5  Açúcar          10    1.5

Suporte a Formatos (Parquet) - Primeiras linhas:
   Produto  Quantidade  Preco
0   Arroz          10    2.5
1   Arroz          15    2.5
2  Feijão          20    3.0
3  Feijão          30    3.0
4  Açúcar           5    1.5
5  Açú